# **Total Water Level**

In [ ]:
# common
import sys
import os
import os.path as op

# pip
import pandas as pd
import numpy as np
import matplotlib
import plotly.express as px

sys.path.insert(0, op.join(os.getcwd(), '..' ,'..'))

from lib.config import *
from lib.snell_propagation import * 


In [ ]:
ID_wave = 1139
ID_sea_level = 'Samoa'

In [ ]:
p_storage = op.join(os.getcwd(), '..', 'storage')

In [ ]:
waves = pd.read_csv(op.join(p_storage, 'Waves_{0}.csv'.format(ID_wave))).set_index('time')

sea_level = pd.read_csv(op.join(p_storage, 'Sea_level_{0}.csv'.format(ID_sea_level)))

## **Wave Station Location**

In [ ]:
# Add coordinates to visualize location of available stations
lat, lon = [-13.6], [188.4]
df_info = pd.DataFrame({'longitude': lon, 'latitude': lat, 'Info': ['HsTpDir']})  


In [ ]:
fig = px.scatter_mapbox(df_info, 
                        lat="latitude", lon="longitude", hover_data={'Info'}, zoom=6)

fig.update_layout(mapbox_style="stamen-terrain", width=800, height=400,
                  margin={"r":0,"t":0,"l":0,"b":0},
                  showlegend=True,)
fig.show()

## **Total Water Level Calculation**

$TWL = AT + SS + R2$<br>

Runup formula for dissipative beaches (Stockdon et al, 2006)<br>
$R2 = 0.043 \cdot \sqrt{H_{s} \cdot L_{0}}$<br>

where $L_{0}=\frac{g}{2\cdot \pi}\cdot T_{m}^{2}$<br>

$T_{m}=0.7 \cdot T_{p}$

### **Find common period of Waves and Sea Level information**

In [ ]:
idx, com_waves, com_sealevel = np.intersect1d(waves.index.dropna(), sea_level.dropna()['Date'].values, return_indices=True)
waves = waves.loc[idx]
sea_level = sea_level.set_index(['Date']).loc[idx]

### **Downscale Hs to obtain Wave Runup**

<span style="font-family: times, Optima; font-size:11pt; color:black;">
To simplify the downscaling of offshore waves to the coast, we assume straight and parallel bathymetry and apply the Snell's law.
    <span>

`Deep Water Waves`

In [ ]:
bathy_orientation = 20                      # degress North
min_range, max_range = 110, 290
h0 = 1000                                   # offshore depth  
h1 = 10                                     # nearshore depth  
g = 9.81

In [ ]:
waves

In [ ]:
waves.loc[(waves['dir'].values > min_range) & (waves['dir'].values < max_range), 'hs'] = 0 

In [ ]:
waves['theta_0'] = np.zeros(len(waves))
waves['theta_0'] = np.where((waves['dir'] > bathy_orientation) & (waves['dir'] < min_range), waves['dir'] - bathy_orientation, waves['theta_0'])
waves['theta_0'] = np.where(waves['dir'] < bathy_orientation, bathy_orientation - waves['dir'], waves['theta_0'])
waves['theta_0'] = np.where(waves['dir'] > max_range, 360 - waves['dir'] + bathy_orientation, waves['theta_0'])

In [ ]:
H0 = waves['hs']
T = waves['tp']
theta_0 = waves['theta_0']

`Shallow Water Waves`

In [ ]:
out0 = np.array([waves_dispersion(T[i], h0) for i in range(len(T))])
L0, k0, C0 = out0[:,0], out0[:,1], out0[:,2]
Cg0 = 0.5 * C0

out1 = np.array([waves_dispersion(T[i], h1) for i in range(len(T))])
L1, k1, C1 = out1[:,0], out1[:,1], out1[:,2]
Cg1 = cg(L1, T, h1)

theta_1 = theta(C0, C1, theta_0)


In [ ]:
H1 = H0 * Ks(Cg0, Cg1) * Kr(theta_0, theta_1)

# Breaking Condition
Hb = 0.8 * h1

# Goda Breaking Condition
m = 0.01
mask_Goda = Goda(L0, h1, m)

H1_Goda = H1.where(H1 < mask_Goda, np.nan)
H1 = H1.where(H1 < Hb, np.nan)

### Calculate Total Water Level

In [ ]:
waves['hs'] = H1_Goda
waves['tm'] = waves['tp'] * 0.7
waves['L0'] = (9.806 / 2 * np.pi)  * waves['tm']**2
waves['TWL'] =  sea_level['Sea level'] + 0.043 * np.sqrt(H1 * L1)
waves['sea_level'] = sea_level['Sea level']

In [ ]:
#%matplotlib
waves[['sea_level', 'hs', 'tp', 'dir', 'TWL']].plot(
    figsize=(17,8),
    style=['-','-', '-', '.'],
    ms=2,
    color=[colors[0],colors[1],colors[2],colors[3]],
    subplots=True, 
    layout=(5,1),
)
plt.show()

## Storage daily series of TWL in station

In [ ]:
# storage .csv
waves.to_csv(op.join(p_storage, 'TWL_{0}.csv'.format(ID_wave)))